# XML - Quick Start Guide

This is nothing new, but is an area that still mystifies many database professional. After a recent discussion on the use of XML for data exchange between database and application, I thought I would try write a quick start guide on the use of XML with SQL Server.

I plan for this to be a living document, as it is a big subject and I am unlikely to think of everything first time, so please feel free to comment.

## The Basics of XML

XML is a type of text file that is designed to hold, and describe, data to be used by machines, whilst also being human readable. Below is the sample of XML which I will be using in all my examples:

![XML](./Images/xml_orig.png "XML")

XML uses Tags which are named tokens encased in a < and >. Tags are generally present in pairs, with a start tag (<example>) and an end tag (</example>). 

An Element is a pair of matching tags and their contents, which can include data and/or other elements. For example; the <Player> element in the above image contains a <name> and a <position> element. The <name> element, on the other hand, only contains data; the value 'Lee Smith'. If an element has no contents, it can be represented by a single tag element, such as <example />. The Root element is a special element which encases the entire XML document.

Attributes are name value pairs that describe an element, such as the number attribute within the <Player> element. Attributes can only exist within start or empty element tags.

XML represents a data hierarchy, whereby everything within an element belongs to that instance of the element. For example, in the above example there are 2 <team> elements, each containing a list of players that belong specifically to that team.

## XML in SQL

The XML datatype was introduced in SQL 2008. It is basically the nvarchar(max) type, but can only hold valid XML, and exposes a number of XML specific functions. Like other types, XML can be used for both data fields and variables.

## Querying XML

XML variables can be used to hold, and interact with, a single XML document. The following code demonstrates the declaring and populating of an XML variable.

``` Declare @Squads xml

Select @Squads = 
'<teams>
  <team name="Bradford Bulls">
    <Players>
      <Player number="1">
        <name>Lee Smith</name>
        <position>Centre</position>
      </Player>
      <Player number="2">
        <name>Ethan Ryan</name>
        <position>Winger</position>
      </Player>
    </Players>
  </team>
  <team name="Keighley Cougars">
    <Players>
      <Player number="1">
        <name>Ritchie Hawkyard</name>
        <position>Fullback</position>
      </Player>
      <Player number="2">
        <name>Andy Gabriel</name>
        <position>Winger</position>
      </Player>
    </Players>
  </team>
</teams>
' ```


You can view the contents of an XML variable with a simple select statement, just like with any other data type. However; the XML will be presented as a single line hyperlink which, if clicked on, opens up the formatted document in a new SSMS tab.

SQL Server provides several functions for accessing data within the XML documents. Unfortunately these functions look nothing like standard SQL, which is part of the reason why they tend to scare people. Data can be selected from an XML document by using the value function, which requires 2 parameters; an address and a datatype for the result. The address denotes the sequence of Elements, working from the outside in, through which you will pass to get to the value that you want. For example:

```
Select @Squads.value('(teams/team/Players/Player/@number)[1]', 'int') SquadNumber,
             @Squads.value('(teams/team/Players/Player/name)[1]', 'varchar(50)') PlayerName,
             @Squads.value('(teams/team/Players/Player/position)[1]', 'varchar(50)') Position

Select @Squads.value('(teams/team/Players/Player/@number)[2]', 'int') SquadNumber,
             @Squads.value('(teams/team/Players/Player/name)[2]', 'varchar(50)') PlayerName,
             @Squads.value('(teams/team/Players/Player/position)[2]', 'varchar(50)') Position
```

Give this a try by running the below code block:

In [ ]:
Declare @Squads xml

Select @Squads = 
'<teams>
  <team name="Bradford Bulls">
    <Players>
      <Player number="1">
        <name>Lee Smith</name>
        <position>Centre</position>
      </Player>
      <Player number="2">
        <name>Ethan Ryan</name>
        <position>Winger</position>
      </Player>
    </Players>
  </team>
  <team name="Keighley Cougars">
    <Players>
      <Player number="1">
        <name>Ritchie Hawkyard</name>
        <position>Fullback</position>
      </Player>
      <Player number="2">
        <name>Andy Gabriel</name>
        <position>Winger</position>
      </Player>
    </Players>
  </team>
</teams>
'

Select @Squads.value('(teams/team/Players/Player/@number)[1]', 'int') SquadNumber,
             @Squads.value('(teams/team/Players/Player/name)[1]', 'varchar(50)') PlayerName,
             @Squads.value('(teams/team/Players/Player/position)[1]', 'varchar(50)') Position

Select @Squads.value('(teams/team/Players/Player/@number)[2]', 'int') SquadNumber,
             @Squads.value('(teams/team/Players/Player/name)[2]', 'varchar(50)') PlayerName,
             @Squads.value('(teams/team/Players/Player/position)[2]', 'varchar(50)') Position

In this snippet we are returning details of the first 2 players in the document, with the number in square brackets at the end of the address allowing us to specify which particular instance of a player that we are interested in. It is worth noting that even if there is only one value matching the address, you must stipulate that it is the first, [1], instance that you are interested in. When we are returning the value from within an element we address the name of the element (name and position in the above example), whereas when we want to return the value of an attribute prefix the name of the attribute with @ (@number in the above example).

Obviously, this method of extracting data is very limited, and would be unworkable if the document contained 100 players rather than just 4. Thankfully there is another method which allows you to define collections of sub-documents, or nodes,  which can be queried as a set.

The following snippet returns a full list of all the players:

```
Select    player.value('(../../@name)[1]', 'varchar(50)') TeamName, 
          player.value('(@number)[1]', 'int') SquadNumber,
          player.value('(name)[1]', 'varchar(50)') PlayerName,
          player.value('(position)[1]', 'varchar(50)') Position
from      @Squads.nodes('/teams/team/Players/Player') players(player) 
```

Give this a try by running the below code block:


In [ ]:
Declare @Squads xml

Select @Squads = 
'<teams>
  <team name="Bradford Bulls">
    <Players>
      <Player number="1">
        <name>Lee Smith</name>
        <position>Centre</position>
      </Player>
      <Player number="2">
        <name>Ethan Ryan</name>
        <position>Winger</position>
      </Player>
    </Players>
  </team>
  <team name="Keighley Cougars">
    <Players>
      <Player number="1">
        <name>Ritchie Hawkyard</name>
        <position>Fullback</position>
      </Player>
      <Player number="2">
        <name>Andy Gabriel</name>
        <position>Winger</position>
      </Player>
    </Players>
  </team>
</teams>
'

Select    player.value('(../../@name)[1]', 'varchar(50)') TeamName, 
          player.value('(@number)[1]', 'int') SquadNumber,
          player.value('(name)[1]', 'varchar(50)') PlayerName,
          player.value('(position)[1]', 'varchar(50)') Position
from      @Squads.nodes('/teams/team/Players/Player') players(player) 

The nodes function defines an address, from which every matching element, and its contents, will be extracted as a sub-document. The part at the end of the From statement defines a virtual table to represent the nodes, in this case a players table containing a player field, of type XML. In the above example there are 4 <Player> elements returned. The value functions, in this case, address items within the sub documents, not the original document. For the TeamName I have addressed the special .. operator, which allows me to step up levels to query the @name attribute of the <team> element.

It is even possible to define nodes based on other nodes. In the following example I extract a collection of teams, and from that collection extract a collection of players. 

``` 
Select team.value('(@name)[1]', 'varchar(50)') TeamName,
       player.value('(@number)[1]', 'int') SquadNumber,
       player.value('(name)[1]', 'varchar(50)') PlayerName,
       player.value('(position)[1]', 'varchar(50)') Position
from   @Squads.nodes('/teams/team') teams(team)
cross apply team.nodes('Players/Player') players(player) 
```

Give this a try by running the below code block:

In [ ]:
Declare @Squads xml

Select @Squads = 
'<teams>
  <team name="Bradford Bulls">
    <Players>
      <Player number="1">
        <name>Lee Smith</name>
        <position>Centre</position>
      </Player>
      <Player number="2">
        <name>Ethan Ryan</name>
        <position>Winger</position>
      </Player>
    </Players>
  </team>
  <team name="Keighley Cougars">
    <Players>
      <Player number="1">
        <name>Ritchie Hawkyard</name>
        <position>Fullback</position>
      </Player>
      <Player number="2">
        <name>Andy Gabriel</name>
        <position>Winger</position>
      </Player>
    </Players>
  </team>
</teams>
'

Select team.value('(@name)[1]', 'varchar(50)') TeamName,
       player.value('(@number)[1]', 'int') SquadNumber,
       player.value('(name)[1]', 'varchar(50)') PlayerName,
       player.value('(position)[1]', 'varchar(50)') Position
from   @Squads.nodes('/teams/team') teams(team)
cross apply team.nodes('Players/Player') players(player) 

The following example demonstrates querying XML fields rather than variables;

In [ ]:
Declare @Squads xml

Select @Squads = 
'<teams>
  <team name="Bradford Bulls">
    <Players>
      <Player number="1">
        <name>Lee Smith</name>
        <position>Centre</position>
      </Player>
      <Player number="2">
        <name>Ethan Ryan</name>
        <position>Winger</position>
      </Player>
    </Players>
  </team>
  <team name="Keighley Cougars">
    <Players>
      <Player number="1">
        <name>Ritchie Hawkyard</name>
        <position>Fullback</position>
      </Player>
      <Player number="2">
        <name>Andy Gabriel</name>
        <position>Winger</position>
      </Player>
    </Players>
  </team>
</teams>
'


Declare   @SquadsTable table ( TestXML xml)

insert    @SquadsTable (TestXML)
values    (@Squads)

Select    team.value('(@name)[1]', 'varchar(50)') TeamName,
          player.value('(@number)[1]', 'int') SquadNumber,
          player.value('(name)[1]', 'varchar(50)') PlayerName,
          player.value('(position)[1]', 'varchar(50)') Position
from      @SquadsTable
cross apply TestXML.nodes('/teams/team') teams(team)
cross apply team.nodes('Players/Player') players(player)

## Generating XML Result Sets

As well was storing and querying XML, SQL Server can generate XML from result sets. You do this by adding the FOR XML keywords to the end of your query. For the demonstration I will use the following table as my source:

```
Create table #Squad
(
TeamName varchar(50),
SquadNumber int,
PlayerName varchar(50),
Position varchar(50)
)

insert #Squad
values ('Bradford Bulls',1,'Lee Smith','Centre'),
       ('Bradford Bulls',2,'Ethan Ryan','Winger'),
       ('Keighley Cougars',1,'Ritchie Hawkyard','Fullback'),
       ('Keighley Cougars',2,'Andy Gabriel','Winger')
```

My aim is to recreate my original XML document from this table. The following code demonstrates creating a simple XML document:

```
Select    distinct TeamName as '@name',
          SquadNumber as 'Players/Player/@number',
          PlayerName as 'Players/Player/name',
          Position as 'Players/Player/position'
from      #Squad
for XML Path('team'), Root('teams')
```

Give this a try by running the below code block:

In [ ]:
Create table #Squad
(
TeamName varchar(50),
SquadNumber int,
PlayerName varchar(50),
Position varchar(50)
)

insert #Squad
values ('Bradford Bulls',1,'Lee Smith','Centre'),
       ('Bradford Bulls',2,'Ethan Ryan','Winger'),
       ('Keighley Cougars',1,'Ritchie Hawkyard','Fullback'),
       ('Keighley Cougars',2,'Andy Gabriel','Winger')
;

Select    distinct TeamName as '@name',
          SquadNumber as 'Players/Player/@number',
          PlayerName as 'Players/Player/name',
          Position as 'Players/Player/position'
from      #Squad
for XML Path('team'), Root('teams')
;

Drop table #Squad

The optional Root keyword allows you to define an element to encompass the entire document, and the Path keyword defined a parent element for your fields. Each field is aliased with its address relative to the Path element. You will notice that the resulting XML differs from the original, in that rather than having 1 element per team, there is a matching element encompassing each player. This is still valid XML, and will produce the same results with our example queries, but is not quite what I wanted.

![XML](./Images/xml-2_orig.png "XML")

Unfortunately producing the desired XML is not quite so straight forward, as specifying collections of elements, such as players, within a single element requires nesting. Nesting involves defining correlated sub queries to  define each collection.

```
Select  TeamName as '@name',
        (       
        Select  SquadNumber as '@number',
                (         
                Select    PlayerName as 'name',
                            Position as 'position'
                from      #Squad
                where     TeamName = p.TeamName
                and       SquadNumber = p.SquadNumber
                For XML Path (''), Type
                )
        from      #Squad p
        where     TeamName = s.TeamName
        for XML Path('Player'), Root('Players'), Type
        )
from    (
        Select distinct  TeamName
        from   #Squad
        ) s
for XML Path('team'), Root('teams')
```

In this example I want each team to appear only once rather than be duplicated, so I am using an inline view to start with a distinct list of teams. This list has a defined Root (teams) and Path (team) to define the outer 2 elements, and a @name attribute.

Within each team we have a sub-query which defines a collection of squad numbers, with a Root (Players), a Path (Player) and an attribute of @number. You may notice that I am also using the Type keyword, as by default SQL will interpret the sub query as text, and remove the < and > characters as they are not allowed in values. Using the Type keyword forces SQL to treat the value as XML and not modify it.

Within each squad number element we have a sub query defining the player information. This time we do not want to encompass these elements within a parent element, and so I specify an empty string for the path.

![XML](./Images/xml-3_orig.png "XML")

Give this a try:




In [ ]:
Create table #Squad
(
TeamName varchar(50),
SquadNumber int,
PlayerName varchar(50),
Position varchar(50)
)

insert #Squad
values ('Bradford Bulls',1,'Lee Smith','Centre'),
       ('Bradford Bulls',2,'Ethan Ryan','Winger'),
       ('Keighley Cougars',1,'Ritchie Hawkyard','Fullback'),
       ('Keighley Cougars',2,'Andy Gabriel','Winger')
;

Select  TeamName as '@name',
        (       
        Select  SquadNumber as '@number',
                (         
                Select    PlayerName as 'name',
                            Position as 'position'
                from      #Squad
                where     TeamName = p.TeamName
                and       SquadNumber = p.SquadNumber
                For XML Path (''), Type
                )
        from      #Squad p
        where     TeamName = s.TeamName
        for XML Path('Player'), Root('Players'), Type
        )
from    (
        Select distinct  TeamName
        from   #Squad
        ) s
for XML Path('team'), Root('teams')
;

Drop table #Squad